# Barrier swaption

In this section we consider Monte Carlo evaluation of a knock-out swaption
under the LMM. We use the knock-out swaption as a guide in our exposition,
its treatment is rather general and it can be used to value diﬀerent barrier
options, where the underlying and barrier can be expressed as functionals of
some diﬀusion process.

A European payer (receiver) swaption is an option that gives its holder a
right, but not an obligation, to enter a payer (receiver) swap at a future date
at a given fixed rate K. Usually, the swaption maturity coincides with the first
reset date T0 of the underlying swap. The underlying swap length TN− T0 is
called the tenor of the swaption.

$$
V_{\text{swaption}}(0) = P(0, T_0) \, \mathbb{E}^{\mathbb{Q}^{T_0}} \left[
\delta \left( R_{\text{swap}}(T_0) - K \right)_+
\sum_{j=1}^{N} P(T_0, T_j) \, \chi(\theta > T_0)
\right]
\tag{6.1}
$$

where $\theta$ is the first exit time of the process $R_{\text{swap}}(s), s \geq 0$,
from the interval $(0, R_{\text{up}})$.
That is, $\theta$ is the earliest time at which the swap rate crosses the upper barrier $R_{\text{up}}$,
causing the knock-out feature of the swaption to be triggered.

In [22]:
import numpy as np
from scipy.linalg import cholesky
from scipy.optimize import minimize
from tqdm import tqdm

## 1. Swap Rate Function

The swap rate $R_{swap}(s)$ can be expressed in terms of the spanning LIBOR rates as:

$$
R_{\text{swap}}(s) = \frac{1 - \frac{1}{\prod\limits_{j=0}^{N-1} \left(1 + \delta L^j(s)\right)}}
{\delta \sum\limits_{i=0}^{N-1} \frac{1}{\prod\limits_{j=0}^{i} \left(1 + \delta L^j(s)\right)}}.
\tag{6.2}
$$

In [23]:
def R_swap(logL, delta):
    L = np.exp(logL)
    discount_prod = np.prod(1 + delta * L)
    numer = 1 - 1.0 / discount_prod
    denom = delta * np.sum(1.0 / np.cumprod(1 + delta * L))
    return numer / denom

## 2. Projection sur la surface barrière

For completeness of the exposition, let us discuss how the projection $\ln L_k^{\pi}$
can be simulated before we return to the description of the algorithm. The
problem of finding point $\ln L_k^{\pi}$ is equivalent to finding the minimum value of the function

$$
\left| \ln L_k^\pi - \ln L_k \right|^2 = \left( \ln L_k^{\pi,0} - \ln L_k^0 \right)^2 + \cdots + \left( \ln L_k^{\pi,N-1} - \ln L_k^{N-1} \right)^2 \tag{6.11}
$$
subject to the constraint
$$
\ln \left(
\frac{ \displaystyle\prod_{j=0}^{N-1} \left( 1 + \delta L_k^{\pi,j} \right) - 1 }{
\delta \left( 1 + \displaystyle\sum_{i=0}^{N-2} \prod_{j=i+1}^{N-1} \left( 1 + \delta L_k^{\pi,j} \right) \right) }
\right) = \ln R_{\text{up}} \tag{6.12}
$$

In [24]:
def project_to_barrier(x, delta, Rup):
    cons = {
        'type': 'eq',
        'fun': lambda logL: np.log(R_swap(logL, delta)) - np.log(Rup)
    }
    res = minimize(
        fun=lambda logL: np.sum((logL - x)**2),
        x0=x.copy(),
        constraints=cons,
        method='SLSQP'
    )
    if not res.success:
        raise RuntimeError("Projection failed: " + res.message)
    return res.x

Hence the minimization problem is reduced to finding the point $\ln L_k^{\pi,1}, \ldots, \ln L_k^{\pi,N-1}$ at which the function $|\ln L_k^{\pi} - \ln L_k|^2$ from (6.11) with $\ln L_k^{\pi,0}$ from (6.13) has its minimum value. This optimization problem can be solved using standard procedures, e.g. the MATLAB function lsqnonlin().


The equation (6.13) is given by:
$$
\ln L_k^{\pi,0} = \ln \left( \frac{
R_{up} \cdot \left( 1 + \sum_{i=0}^{N-2} \prod_{j=i+1}^{N-1} \left( 1 + \delta L_k^{\pi,j} \right) \right) + 1
}{
\prod_{j=1}^{N-1} \left( 1 + \delta L_k^{\pi,j} \right)
} - \frac{1}{\delta} \right) \tag{6.13}
$$

In [45]:
# -- Fast projection to barrier (N-1 unconstrained) --
def project_to_barrier2(x, delta, Rup):
    """
    Projection rapide via élimination analytique de ln L[0] (eq.6.13) et optimisation sur ln L[1..N-1].
    """
    N = x.shape[0]

    def objective(y):
        # y = logL[1..N-1]
        Lpi_tail = np.exp(y)                  # LIBOR rates
        one_plus = 1 + delta * Lpi_tail       # (1+δ·Lpi)
        # produits inverses cumulés: prods[i] = ∏_{j=i..N-2} one_plus[j]
        revcp = np.cumprod(one_plus[::-1])[::-1]
        prods = revcp  # shape (N-1,)
        sum_term = 1 + prods.sum()
        numerator = Rup * sum_term + 1
        denominator = np.prod(one_plus)
        # Formule 6.13 pour ln L0 (soustraction hors du log)
        logL0 = np.log(numerator / denominator) - 1.0 / delta
        # Construire vecteur complet logLpi
        logLpi = np.empty(N)
        logLpi[0] = logL0
        logLpi[1:] = y
        return np.sum((logLpi - x)**2)

    # initialisation et optimisation
    y0 = x[1:].copy()
    res = minimize(objective, y0, method='L-BFGS-B')
    if not res.success:
        raise RuntimeError("Projection2 failed: " + res.message)
    y_opt = res.x

    # Reconstruction finale de logLpi
    Lpi_tail = np.exp(y_opt)
    one_plus = 1 + delta * Lpi_tail
    revcp = np.cumprod(one_plus[::-1])[::-1]
    prods = revcp
    sum_term = 1 + prods.sum()
    numerator = Rup * sum_term + 1
    denominator = np.prod(one_plus)
    logL0 = np.log(numerator / denominator) - 1.0 / delta
    logLpi = np.empty(N)
    logLpi[0] = logL0
    logLpi[1:] = y_opt
    return logLpi

## 3. Setup du modèle

- Nombre de forward rates `N`
- Maturité $(T_0)$ découpée en `M` pas de taille `h`
- Courbe initiale `L0`, volatilités `sigma`
- Corrélation exponentielle $( \rho_{ij} = e^{-\beta|T_i - T_j|} $)

In [27]:
# Paramètres
N     = 10
T0    = 10.0
delta = 1.0
M     = 100
h     = T0 / M
K     = 0.01   # Strike
Rup   = 0.075  # Barrier
L0    = 0.05 * np.ones(N)
sigma = 0.10 * np.ones(N)
beta  = 0.1

$$
\rho_{i,j} = \exp(-\beta |T_i - T_j|).
\tag{3.6}
$$

Where $U$ is the upper triangular matrix
$$
\rho = UU^\top,
\tag{3.5}
$$

In [28]:
# Corrélation et Cholesky
times = np.arange(N) * delta + T0 - N * delta
rho   = np.exp(-beta * np.abs(times[:, None] - times[None, :]))
U     = cholesky(rho, lower=False)

In [29]:
# PRICE conventionnel
P0_T0 = 1.0 / (1 + delta * L0[0])

In [30]:
# Matrice précomputée pour drift (i >= j)
full_mat  = rho * np.outer(sigma, sigma)
drift_mat = np.tril(full_mat)

## 4. Simulation Monte Carlo

- **Coarse check** : test rapide pour rester loin de la barrière
- **Fine check** : perturbation maximale en un pas
- **Boundary zone** : projection + saut auxiliaire

Introduce
$$ \ln L_{k, Max} = \max_i \ln L_k^i $$

$$
\ln \hat{L}_{k+1} = \ln L_{k, Max} + \sigma_{Max}^2 hN - \frac{1}{2} \sigma_{Max}^2 h + \sigma_{Max} \sqrt{hN}, \tag{6.7}
$$

where $$ \sigma_{Max} = \max_{i,k} \sigma_i(t_k).$$
Using the fact that $$ R_{swap}(\hat{L}_{k+1}, …, \hat{L}_{k+1}) = \hat{L}_{k+1} $$

In [31]:
# Nombre de chemins
n_paths    = 100_000
payoffs    = np.zeros(n_paths)
exit_times = np.zeros(n_paths)  # stocke l'étape d'exit pour chaque chemin
# Nouvelle variable pour stocker la somme des discounts P(T0,Tj) pour chemins survivants
discount_sums = np.zeros(n_paths)
prices = np.zeros(n_paths)

In [ ]:
for p in tqdm(range(n_paths), desc="MC paths"):
    # Initialize per-path variables
    logL = np.log(L0)
    knocked_out = False
    exit_step = M
    # Precompute coarse_bound from eq (6.7)
    sigma_max = np.max(sigma)
    coarse_bound = (
        sigma_max**2 * h * N
        - 0.5 * sigma_max**2 * h
        + sigma_max * np.sqrt(h * N)
    )

    for k in range(M):
        L = np.exp(logL)
        # Drift vector
        v         = delta * L / (1 + delta * L)
        drift_vec = drift_mat.dot(v) * h - 0.5 * sigma**2 * h

        # Coarse check: bound from (6.7)
        if np.max(logL) + coarse_bound < np.log(Rup):
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U @ xi)
            logL    += drift_vec + diffusion
            continue

        # Fine check: worst-case drift + diffusion per coordinate
        L_vec = L  # already computed
        idx   = np.arange(N)
        drift_worst = (idx + 1) * sigma * sigma_max * h
        diff_worst  = sigma * np.sqrt((N - idx) * h)
        L_pert      = L_vec * (1 + drift_worst + diff_worst)
        logL_pert   = np.log(L_pert)
        if R_swap(logL_pert, delta) < Rup:
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U @ xi)
            logL    += drift_vec + diffusion
            continue

        # Boundary zone
        logL_pi = project_to_barrier2(logL, delta, Rup)
        dist    = np.linalg.norm(logL_pi - logL)
        lambda_k = np.sqrt(N) * coarse_bound  # use bound as lambda*sqrt(h)

        if np.random.rand() < lambda_k / (dist + lambda_k):
            knocked_out = True
            exit_step   = k
            break

        # Jump back then Euler
        logL    += (lambda_k / dist) * (logL_pi - logL)
        xi        = np.random.choice([-1,1], size=N)
        diffusion = sigma * np.sqrt(h) * (U @ xi)
        logL    += drift_vec + diffusion

    exit_times[p] = exit_step * h
    if not knocked_out:
        payoffs[p] = delta * max(R_swap(logL, delta) - K, 0)
        # L est le vecteur des prix finaux des forwards: L_end = exp(logL)
        L_end = np.exp(logL)
        # discount_curve[j] = P(T0, T_{j+1}) = prod_{i=0..j} 1/(1+delta*L_end[i])
        discount_curve = np.cumprod(1.0/(1 + delta * L_end))
        # somme pour j=1...N
        discount_sums[p] = np.sum(discount_curve)
        prices[p] = P0_T0 * (discount_sums[p] * payoffs[p])

MC paths: 100%|██████████| 100000/100000 [03:38<00:00, 457.59it/s]


# Parallélisation

## 5. Résultat

Calcul du prix par moyenne des payoffs et actualisation

In [47]:
price           = np.mean(prices)
mean_exit_time  = np.mean(exit_times)
print(f"Barrier swaption price:    {price:.6f}")
print(f"Mean exit time (in years): {mean_exit_time:.4f}")

Barrier swaption price:    0.265705
Mean exit time (in years): 9.3058


## 6. Approximation analytique

Pour comparaison, sous le **Swap Market Model (SMM)**, on dispose d'une solution fermée analogue à l'équation (4.3) pour la swaption à barrière :


```
V_swaption_SMM(0) = δ ∑_{j=1}^N P(0,T_j) { R_swap(0)[Φ(δ₊(R_swap(0)/K, v)) - Φ(δ₊(R_swap(0)/R_up, v))]
                                   - K[Φ(δ₋(R_swap(0)/K, v)) - Φ(δ₋(R_swap(0)/R_up, v))] }
```

avec
```
δ₊(x,v) = (ln x + v²/2)/v,   δ₋(x,v) = (ln x - v²/2)/v,
 v² ≈ ∑_{i,j} ω_i(0) ω_j(0) L_i(0)L_j(0) ρ_{ij} ∫₀^{T0} σ_i(s)σ_j(s) ds
```

Cette approximation sert de référence pour évaluer la cohérence de la simulation LMM.

In [120]:
from scipy.stats import norm

In [191]:
# Swap rate initial R0
R0 = R_swap(np.log(L0), delta)

# --- 4) Discount factors P(0,Tj) pour j=1..N ---
P0_T = np.concatenate(([1.0], np.cumprod(1.0 / (1 + delta * L0))))
annuity = np.sum(P0_T[1:])   # somme des P(0,Tj)

# --- 5) Pondérations ω_i(0) vectorisées ---
prefix = np.cumprod(1 + delta * L0)            # Π_{j=0..i}(1+δ L0[j])
denom_w = delta * np.sum(1.0 / prefix)
weights = (1.0 - 1.0 / prefix) / denom_w       # shape (N,)

# --- 6) Volatilité effective v selon Rebonato (6.5) ---
# ∫₀^T0 σ_i(s)σ_j(s) ds = σ_i * σ_j * T0
integrals = np.outer(sigma, sigma) * T0         # matrice (i,j)
wL = weights * L0                              # vecteur (i)
num = np.outer(wL, wL) * rho * integrals        # numérateur v²
v2  = num.sum() / (R0**2)
v   = np.sqrt(v2)

# --- 7) Fonctions d± ---
dplus  = lambda x: (np.log(x) + 0.5 * v**2) / v
dminus = lambda x: (np.log(x) - 0.5 * v**2) / v

# --- 8) Ratios clé ---
x1 = R0 / K
x2 = R0 / Rup
y1 = Rup**2 / (K * R0)
y2 = Rup / R0

# --- 9) Termes de la formule fermée (6.4) ---
term1 = R0 * (norm.cdf(dplus(x1)) - norm.cdf(dplus(x2)))
term2 = - norm.cdf(dplus(x2))
term3 = - K   * (norm.cdf(dminus(x1)) - norm.cdf(dminus(x2)))
term4 = - Rup * (norm.cdf(dplus(y1)) - norm.cdf(dplus(y2)))
term5 =   K*R0 * (norm.cdf(dminus(y1)) - norm.cdf(dminus(y2)))

# --- 10) Prix analytique final ---
price_analytic = delta * annuity * (term1 + term2 + term3 + term4 + term5)

print(f"Swap rate initial R0:         {R0:.6f}")
print(f"Volatilité effective v:       {v:.6f}")
print(f"Prix analytique barrière SMM: {price_analytic:.6f}")

Swap rate initial R0:         0.050000
Volatilité effective v:       0.081966
Prix analytique barrière SMM: 0.308865


# 7. Comparaison avec méthode MC 1/2 ordre faible

In [54]:
for p in tqdm(range(n_paths), desc="MC paths"):
    logL        = np.log(L0)
    knocked_out = False
    exit_step   = M

    # pré-calcul constant
    sigma_max   = np.max(sigma)
    coarse_bound = (
        sigma_max**2 * h * N
        - 0.5 * sigma_max**2 * h
        + sigma_max * np.sqrt(h * N)
    )

    for k in range(M):
        L         = np.exp(logL)
        # drift Euler faible
        v         = delta * L / (1 + delta * L)
        drift_vec = drift_mat.dot(v) * h - 0.5 * sigma**2 * h

        # 1) Coarse check : si loin de la barrière
        if np.max(logL) + coarse_bound < np.log(Rup):
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U @ xi)
            logL    += drift_vec + diffusion
            continue

        # 2) Fine check : worst-case drift+diffusion par composante
        idx         = np.arange(N)
        drift_worst = (idx + 1) * sigma * sigma_max * h
        diff_worst  = sigma * np.sqrt((N - idx) * h)
        L_pert      = L * (1 + drift_worst + diff_worst)
        logL_pert   = np.log(L_pert)
        if R_swap(logL_pert, delta) < Rup:
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U @ xi)
            logL    += drift_vec + diffusion
            continue

        # 3) Zone frontière : projection & stop (ordre 1/2)
        logL        = project_to_barrier2(logL, delta, Rup)
        exit_step   = k
        knocked_out = True
        break

    exit_times[p] = exit_step * h

    if not knocked_out:
        payoffs[p]     = delta * max(R_swap(logL, delta) - K, 0)
        discounts      = np.cumprod(1.0 / (1 + delta * np.exp(logL)))
        discount_sums[p] = np.sum(discounts)
        prices[p]       = P0_T0 * discount_sums[p] * payoffs[p]

MC paths: 100%|██████████| 100000/100000 [03:14<00:00, 512.99it/s]


In [57]:
price           = np.mean(prices)
mean_exit_time  = np.mean(exit_times)
print(f"Barrier swaption price:    {price:.6f}")
print(f"Mean exit time (in years): {mean_exit_time:.4f}")

Barrier swaption price:    0.271849
Mean exit time (in years): 9.2630


# 8. Variance reduction